In [1]:
import numpy as np
from collections import Counter

# Sample dataset
dataset = [
    ['Sunny', 'Hot', 'High', False, 'No'],
    ['Sunny', 'Hot', 'High', True, 'No'],
    ['Overcast', 'Hot', 'High', False, 'Yes'],
    ['Rain', 'Mild', 'High', False, 'Yes'],
    ['Rain', 'Cool', 'Normal', False, 'Yes'],
    ['Rain', 'Cool', 'Normal', True, 'No'],
    ['Overcast', 'Cool', 'Normal', True, 'Yes'],
    ['Sunny', 'Mild', 'High', False, 'No'],
]

header = ['Outlook', 'Temperature', 'Humidity', 'Windy', 'PlayTennis']

# Helper: calculate gini
def gini_index(groups, classes):
    total_instances = sum([len(group) for group in groups])
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        labels = [row[-1] for row in group]
        for class_val in classes:
            p = labels.count(class_val) / size
            score += p ** 2
        gini += (1.0 - score) * (size / total_instances)
    return gini

# Split data
def split_data(index, value, dataset):
    left, right = [], []
    for row in dataset:
        if row[index] == value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Best split
def get_best_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    best_index, best_value, best_score, best_groups = None, None, float('inf'), None
    for index in range(len(dataset[0]) - 1):
        values = set(row[index] for row in dataset)
        for value in values:
            groups = split_data(index, value, dataset)
            gini = gini_index(groups, class_values)
            if gini < best_score:
                best_index, best_value, best_score, best_groups = index, value, gini, groups
    return {'index': best_index, 'value': best_value, 'groups': best_groups}

# Leaf node (majority class)
def to_leaf(group):
    outcomes = [row[-1] for row in group]
    return Counter(outcomes).most_common(1)[0][0]

# Recursive split
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])

    # terminal nodes
    if not left or not right:
        node['left'] = node['right'] = to_leaf(left + right)
        return

    if depth >= max_depth:
        node['left'], node['right'] = to_leaf(left), to_leaf(right)
        return

    if len(left) <= min_size:
        node['left'] = to_leaf(left)
    else:
        node['left'] = get_best_split(left)
        split(node['left'], max_depth, min_size, depth + 1)

    if len(right) <= min_size:
        node['right'] = to_leaf(right)
    else:
        node['right'] = get_best_split(right)
        split(node['right'], max_depth, min_size, depth + 1)

# Build the tree
def build_tree(train, max_depth, min_size):
    root = get_best_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Print tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[%s = %s]' % ((depth*'  ', header[node['index']], node['value'])))
        print_tree(node['left'], depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('%s-> %s' % ((depth*'  '), node))

# Build and print the tree
tree = build_tree(dataset, max_depth=3, min_size=1)
print_tree(tree)


[Outlook = Sunny]
  [Outlook = Sunny]
    -> No
    -> No
  [Windy = False]
    [Outlook = Rain]
      -> Yes
      -> Yes
    [Outlook = Rain]
      -> No
      -> Yes
